In [1]:
import json
from enum import Enum
from collections import namedtuple

In [20]:
class events_list(Enum): 
    Write = 1
    Read = 2
    Acquire = 3
    Release = 4

In [125]:
def _json_object_hook(d): 
    return namedtuple('event_raw', d.keys())(*d.values())

def json2obj(data): 
    return json.loads(data, object_hook=_json_object_hook)

class event_raw:
    def __init__(self):
        self.tid = -1
        self.e_type = 0
        self.var_name = ""
        self.vc = []
        self.vc_r = []
        self.vc_w = []

In [322]:
class critical_section:
    def __init__(self,thread_num):
        self.locks = set()
        self.objects = set()
        self.child_sections = []
        self.tid = thread_num

    def __str__(self):
        ret_str = "Thread : %d \nLocks : %s \nObjects : %s \nChild Sections :\n" % (self.tid,self.locks,self.objects)
        for child_cs in self.child_sections:
            ret_str= ret_str+str(child_cs)
        return ret_str
    
    def clear_lists(self):
        self.locks.clear()
        self.objects.clear()
        self.child_sections.clear()
    
    def create_child_Section(self,evt_obj,thrd_stk):
        temp = self
        size_stk = len(thrd_stk)
        while size_stk-1 > 1:
            if len(temp.child_sections) > 0:
                temp.child_sections[-1].locks.add(evt_obj.var_name)
                temp = temp.child_sections[-1]
            size_stk=size_stk-1
        temp.child_sections.append(critical_section(evt_obj.tid))
        temp.child_sections[-1].locks.add(evt_obj.var_name)
    
    def operate_child(self,evt_obj,thrd_stk):
        temp = self
        size_stk = len(thrd_stk)
        while size_stk-1 > 0:
            temp = temp.child_sections[-1]
            size_stk=size_stk-1
            temp.objects.add(evt_obj.var_name)

In [323]:
# event_log_raw = open("../Figure1.log")
event_log_raw = open("temp.log")
event_log = json.loads(event_log_raw.read())

In [324]:
events = []
for line in event_log:
    events.append(json2obj(json.dumps(line)))

In [329]:
thread_cs = []
critical_section_list = []
thread_lock_stack = []

for x in events:
    if(len(thread_cs)<x.tid):
        while(len(thread_cs)!=x.tid):
            thread_cs.append(None)
            thread_lock_stack.append([])
    
    if thread_cs[x.tid-1]==None: # Start of new crititcal section
        print("Starting new critial section : ")
        thread_cs[x.tid-1] = critical_section(x.tid)
        thread_cs[x.tid-1].clear_lists()
        if x.e_type == events_list.Read.value or x.e_type == events_list.Write.value:
            thread_cs[x.tid-1].objects.add(x.var_name)
        elif x.e_type == events_list.Acquire.value:
            thread_cs[x.tid-1].locks.add(x.var_name)
            thread_lock_stack[x.tid-1].append(x.var_name)
    else:
        if x.e_type == events_list.Read.value or x.e_type == events_list.Write.value:
            if len(thread_lock_stack[x.tid-1]) > 1:
                thread_cs[x.tid-1].operate_child(x,thread_lock_stack[x.tid-1])
            thread_cs[x.tid-1].objects.add(x.var_name)
        elif x.e_type == events_list.Acquire.value:
            if thread_lock_stack[x.tid-1][-1] != x.var_name:
                thread_lock_stack[x.tid-1].append(x.var_name)
                thread_cs[x.tid-1].create_child_Section(x,thread_lock_stack[x.tid-1])
            thread_cs[x.tid-1].locks.add(x.var_name)
        elif x.e_type == events_list.Release.value:
            if len(thread_lock_stack[x.tid-1]) == 1:
                print(thread_cs[x.tid-1])
                critical_section_list.append(thread_cs[x.tid-1])
                thread_cs[x.tid-1] = None
            # Note: Optimistic assumption, top of the lock is same
            if thread_lock_stack[x.tid-1][-1] == x.var_name:
#                 print("Lock Stack before pop : "+str(thread_lock_stack[x.tid-1])+"\n")
                thread_lock_stack[x.tid-1].pop()
#     print(thread_cs[x.tid-1])

Starting new critial section : 
Thread : 1 
Locks : {'l', 'n', 'm'} 
Objects : {'xl', 'x', 'b', 'a', 'y', 'yl'} 
Child Sections :
Thread : 1 
Locks : {'l', 'n'} 
Objects : {'xl', 'x', 'yl', 'y'} 
Child Sections :
Thread : 1 
Locks : {'n'} 
Objects : {'xl', 'yl'} 
Child Sections :
Thread : 1 
Locks : {'n'} 
Objects : {'xl', 'yl'} 
Child Sections :



In [330]:
for cs in critical_section_list:
    print(cs)

Thread : 1 
Locks : {'l', 'n', 'm'} 
Objects : {'xl', 'x', 'b', 'a', 'y', 'yl'} 
Child Sections :
Thread : 1 
Locks : {'l', 'n'} 
Objects : {'xl', 'x', 'yl', 'y'} 
Child Sections :
Thread : 1 
Locks : {'n'} 
Objects : {'xl', 'yl'} 
Child Sections :
Thread : 1 
Locks : {'n'} 
Objects : {'xl', 'yl'} 
Child Sections :

